In [1]:
import logging
import time
import random
from datetime import datetime

n_campi = 'CREC'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('consolidação')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_consolidacao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {n_campi}')
# logger.debug('debug message')
# logger.warning('warn message')

In [2]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {n_campi}')
levantamento = pd.read_excel('consolidacao/levantamento/2024_levantamento_CREC.xlsx')

logger.info(f'carregando palnilha de estoque suap campus {n_campi}')
suap = pd.read_excel('consolidacao/suap/2024-01-17_16-36_CREC_relatorio_almox.xlsx')

logger.info(f'tratando dados da planilha de levantameto fisico {n_campi}')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]


# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

# print(levantamento.info())
# print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})

# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

logger.info(f'removendo dados duplicados')
# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

logger.info(f'cruzando dados entre levantamento fisico e suap')

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
# display(suap_cross)
# display(levantamento_cross)

logger.info(f'consolidando dados')

logger.info(f'exportando planilha de consolidação')
consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

logger.info(f'consolidação campus {n_campi} finalizada')

In [3]:
logger.info(f'montando planilha de ajustes ')

consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

logger.info(f'exportando planilha de ajustes ')

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.07.1338,"AÇÚCAR, TIPO CRISTAL, CARACTERÍSTICAS ADICIONA...",KG,15.0,07 - GENEROS DE ALIMENTACAO,"AÇÚCAR, TIPO CRISTAL, CARACTERÍSTICAS ADICIONA...",15.0,calm@recife.ifpe.edu.br,both,0.0
1,30.07.67,AÇUCAR CRISTAL EMBALAGEM C/ 1 KG.,KG,1286.0,07 - GENEROS DE ALIMENTACAO,AÇUCAR CRISTAL EMBALAGEM C/ 1 KG.,1286.0,calm@recife.ifpe.edu.br,both,0.0
2,30.11.371,HIPOCLORITO DE CALCIO GRANULADO A 65%,Kg,40.0,11 - MATERIAL QUIMICO,HIPOCLORITO DE CALCIO GRANULADO A 65%,40.0,calm@recife.ifpe.edu.br,both,0.0
3,30.16.1046,"CARTOLINA, MATERIAL CELULOSE VEGETAL, GRAMATUR...",UN,79.0,16 - MATERIAL DE EXPEDIENTE,"CARTOLINA, MATERIAL CELULOSE VEGETAL, GRAMATUR...",79.0,calm@recife.ifpe.edu.br,both,0.0
4,30.16.1052,PAPEL LINHO ( COR BRANCA ) - GRAMATURA 180 G/M...,CAIXA,43.0,16 - MATERIAL DE EXPEDIENTE,PAPEL LINHO ( COR BRANCA ) - GRAMATURA 180 G/M...,43.0,calm@recife.ifpe.edu.br,both,0.0
...,...,...,...,...,...,...,...,...,...,...
714,30.42.700,GROSA,UN,6.0,42 - FERRAMENTAS,GROSA,6.0,calm@recife.ifpe.edu.br,both,0.0
715,30.42.785,"BANDEJA PINTURA, MATERIAL PLÁSTICO, COMP. 24,5...",UN,18.0,42 - FERRAMENTAS,"BANDEJA PINTURA, MATERIAL PLÁSTICO, COMP. 24,5...",18.0,calm@recife.ifpe.edu.br,both,0.0
716,30.42.786,"BANDEJA PINTURA, MATERIAL PLÁSTICO, COMP. 29 C...",UN,14.0,42 - FERRAMENTAS,"BANDEJA PINTURA, MATERIAL PLÁSTICO, COMP. 29 C...",14.0,calm@recife.ifpe.edu.br,both,0.0
717,30.42.981,"BROCA, MATERIAL AÇO RÁPIDO, DIÂMETRO 3 MM, TAM...",UN,7.0,42 - FERRAMENTAS,"BROCA, MATERIAL AÇO RÁPIDO, DIÂMETRO 3 MM, TAM...",7.0,calm@recife.ifpe.edu.br,both,0.0


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.07.1338,07 - GENEROS DE ALIMENTACAO,"AÇÚCAR, TIPO CRISTAL, CARACTERÍSTICAS ADICIONA...",15.0,15.0,0.0,adequado
1,30.07.67,07 - GENEROS DE ALIMENTACAO,AÇUCAR CRISTAL EMBALAGEM C/ 1 KG.,1286.0,1286.0,0.0,adequado
2,30.11.371,11 - MATERIAL QUIMICO,HIPOCLORITO DE CALCIO GRANULADO A 65%,40.0,40.0,0.0,adequado
3,30.16.1046,16 - MATERIAL DE EXPEDIENTE,"CARTOLINA, MATERIAL CELULOSE VEGETAL, GRAMATUR...",79.0,79.0,0.0,adequado
4,30.16.1052,16 - MATERIAL DE EXPEDIENTE,PAPEL LINHO ( COR BRANCA ) - GRAMATURA 180 G/M...,43.0,43.0,0.0,adequado
...,...,...,...,...,...,...,...
713,30.42.7,42 - FERRAMENTAS,BROCA AÇO RÁPIDO 3/8,3.0,1.0,2.0,pedido no suap
714,30.42.700,42 - FERRAMENTAS,GROSA,6.0,6.0,0.0,adequado
715,30.42.785,42 - FERRAMENTAS,"BANDEJA PINTURA, MATERIAL PLÁSTICO, COMP. 24,5...",18.0,18.0,0.0,adequado
716,30.42.786,42 - FERRAMENTAS,"BANDEJA PINTURA, MATERIAL PLÁSTICO, COMP. 29 C...",14.0,14.0,0.0,adequado
